# Datacube extension testing in the Resource Catalogue

## Introduction

This notebook is used for testing datacube support in the Resource Catalogue. \
Eventually, it will be integrated into the EOEPCA notebook 04 Resource Catalogue. \

## Description

The notebook is divided into several sections to make it easier to understand.



## Section 1: Imports and EOEPCA connection setup
This section includes importing everything from the 04 notebook. \
It also includes the initial code blocks as these are used to setup the domain variables. \
Some information related to csw and the domain is printed as a check.

In [ ]:
from owslib.csw import CatalogueServiceWeb
from owslib.ogcapi.records import Records
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, BBox, SortBy, SortProperty
from geolinks import sniff_link


In [ ]:
# We use the develop domain as it has Datacube examples 

#base_domain = "demo.eoepca.org"
base_domain = "develop.eoepca.org"
#base_domain = "185.52.193.87.nip.io"
#workspace_prefix = "demo-user"
workspace_prefix = "user"

In [ ]:
system_catalogue_endpoint = f'https://resource-catalogue.{base_domain}/csw'
print(system_catalogue_endpoint)

csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)

print(csw.identification.type)
print(csw.version)
print([op.name for op in csw.operations])

In [ ]:
csw.get_operation_by_name('GetRecords').constraints

## Section 2: Discovering Datacubes

The second section involves some code to find and identify the presence of datacubes.\
A couple of datacube examples should be present on the develop cluster.\
These will later be accessed using different methods. 

Currently there are 3 examples available on the EOEPCA develop cluster:

[PIC 1 Month 9km](https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items/PIC_1-month_9km)

[BICEP_NCEO_PIC_ESA-OC-L3S-OC](https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items/BICEP_NCEO_PIC_ESA-OC-L3S-OC_PRODUCTS-MERGED-1M_MONTHLY_9km_GEO_PML_OCx_QAA-199711-fv4.2.nc)

[ESACCI-LC-L4-LCCS-Map-300m-P1Y-2009](https://resource-catalogue.develop.eoepca.org/collections/metadata:main/items/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2009-v2.0.7cds)

We will use the OGC CSW interface, building an OGC query and execute through the OWSLib client:

In [ ]:
bbox_query = BBox([-180, -90, 180, 90])

#begin = PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2009-01-10 00:00')

#end = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal='2009-12-31 00:00')

title = PropertyIsEqualTo(propertyname='apiso:Title', literal='ESA CCI Land Cover Map for 2009')


In [ ]:
filter_list=[
    And(
        [
           bbox_query,
           title,
        ]
    )
]

In [ ]:
csw.getrecords2(constraints=filter_list)
csw.results

We can then get information of all the query results

In [ ]:
item_identifier = list(csw.records)[0]
print(item_identifier)

In [ ]:
record = csw.records[item_identifier]

print("Title:", record.title)
print("Abstract:", record.abstract)
print("Date:", record.date)
print("Type:", record.type)
print("Coverate:", record.coverage)
print("Format", record.format)
print("License:", record.license)
print("Source:", record.source)
print("Spatial:", record.spatial)


bbox = record.bbox
print("Bounding Box Dimensions:")
print("Min X:", bbox.minx)
print("Max X:", bbox.maxx)
print("Min Y:", bbox.miny)
print("Max Y:", bbox.maxy)

We will now use the OGC API - Records interface to work with the JSON data:

In [ ]:
records_catalogue_endpoint = f'https://resource-catalogue.{base_domain}'
w = Records(records_catalogue_endpoint)

In [ ]:
w.conformance()

In [ ]:
w.collection_queryables('metadata:main')

We just query the catalogue for all records in the default collection

In [ ]:
my_catalogue_query = w.collection_items('metadata:main')
my_catalogue_query['numberMatched']

And we show the title of the first record of the catalogue

In [ ]:
my_catalogue_query['features'][0]['properties']['title']

Then we execute a CQL text query for the records that have the datacube extension

In [ ]:
my_catalogue_cql_text_query = w.collection_items('metadata:main', filter="anytext ILIKE '%datacube%'")
my_catalogue_cql_text_query['numberMatched']

In [ ]:
my_catalogue_cql_text_query['features'][0]['properties']['title']

We present the full JSON (STAC) record for the first result

In [ ]:
my_catalogue_cql_text_query['features'][0]